# Connect 4 Minimax w/ Alpha-Beta Pruning
---

Heuristic search algorithm that considers two players:
- MAX seeks the move with the highest likelihood of winning.
- MIN seeks the move that minimizes MAX’s chance of winning.

The algorithm performs a depth-first search (DFS) on a tree of game states to select what the optimal move is at each turn.
Each state of the game is assigned a score, called the utility:
- For Connect 4, the state would represent the board at any particular turn.
- The method of scoring is left to the programmer, for Connect 4 we could consider chunks of the board and count how many pieces in each belong to players, or are blank. 

“Your next move is only as strong as the opponent’s move is weak”


In [ ]:
%pip install pygame
%pip install numpy

In [2]:
import numpy as np
import random
import pygame
import sys
import math
import time

pygame 2.6.1 (SDL 2.28.4, Python 3.10.11)
Hello from the pygame community. https://www.pygame.org/contribute.html


Defining some constants for the game:

In [4]:
BLUE = (0,0,255)
BLACK = (0,0,0)
RED = (255,0,0)
YELLOW = (255,255,0)

ROW_COUNT = 6
COLUMN_COUNT = 7

SQUARESIZE = 100
S_WIDTH = COLUMN_COUNT * SQUARESIZE
S_HEIGHT = (ROW_COUNT+1) * SQUARESIZE
RADIUS = int(SQUARESIZE/2 - 5)

PLAYER = 0 
AI = 1 

EMPTY = 0
PLAYER_PIECE = 1
AI_PIECE = 2

WINDOW_LENGTH = 4

Functions for the game:

In [5]:
def create_board():
	board = np.zeros((ROW_COUNT,COLUMN_COUNT))
	return board

def drop_piece(board, row, col, piece):
	board[row][col] = piece

def is_valid_location(board, col):
	return board[ROW_COUNT-1][col] == 0

def get_next_open_row(board, col):
	for r in range(ROW_COUNT):
		if board[r][col] == 0:
			return r

def print_board(board):
	print(np.flip(board, 0))

def winning_move(board, piece):
	# Check horizontal locations for win
	for c in range(COLUMN_COUNT-3):
		for r in range(ROW_COUNT):
			if board[r][c] == piece and board[r][c+1] == piece and board[r][c+2] == piece and board[r][c+3] == piece:
				return True

	# Check vertical locations for win
	for c in range(COLUMN_COUNT):
		for r in range(ROW_COUNT-3):
			if board[r][c] == piece and board[r+1][c] == piece and board[r+2][c] == piece and board[r+3][c] == piece:
				return True

	# Check positively sloped diaganols
	for c in range(COLUMN_COUNT-3):
		for r in range(ROW_COUNT-3):
			if board[r][c] == piece and board[r+1][c+1] == piece and board[r+2][c+2] == piece and board[r+3][c+3] == piece:
				return True

	# Check negatively sloped diaganols
	for c in range(COLUMN_COUNT-3):
		for r in range(3, ROW_COUNT):
			if board[r][c] == piece and board[r-1][c+1] == piece and board[r-2][c+2] == piece and board[r-3][c+3] == piece:
				return True

def draw_board(board):
	for c in range(COLUMN_COUNT):
		for r in range(ROW_COUNT):
			pygame.draw.rect(SCREEN, BLUE, (c*SQUARESIZE, r*SQUARESIZE+SQUARESIZE, SQUARESIZE, SQUARESIZE))
			pygame.draw.circle(SCREEN, BLACK, (int(c*SQUARESIZE+SQUARESIZE/2), int(r*SQUARESIZE+SQUARESIZE+SQUARESIZE/2)), RADIUS)
	
	for c in range(COLUMN_COUNT):
		for r in range(ROW_COUNT):		
			if board[r][c] == PLAYER_PIECE:
				pygame.draw.circle(SCREEN, RED, (int(c*SQUARESIZE+SQUARESIZE/2), S_HEIGHT-int(r*SQUARESIZE+SQUARESIZE/2)), RADIUS)
			elif board[r][c] == AI_PIECE: 
				pygame.draw.circle(SCREEN, YELLOW, (int(c*SQUARESIZE+SQUARESIZE/2), S_HEIGHT-int(r*SQUARESIZE+SQUARESIZE/2)), RADIUS)
	pygame.display.update()

Methods for implementing Minimax:

In [6]:
def evaluate_window(window, piece):
	score = 0
	opp_piece = PLAYER_PIECE # Constants here for 1, 2 depending on player
	if piece == PLAYER_PIECE:
		opp_piece = AI_PIECE

	# If this window has a winning position for us, give it a very high score
	if window.count(piece) == 4:
		score += 100
	elif window.count(piece) == 3 and window.count(EMPTY) == 1: # Empty is a constant for 0
		score += 5
	elif window.count(piece) == 2 and window.count(EMPTY) == 2:
		score += 2

	# We want to avoid situations where the opponent is close to winning
	if window.count(opp_piece) == 3 and window.count(EMPTY) == 1:
		score -= 4

	return score

def score_position(board, piece):
	score = 0

	# Score center column
	center_array = [int(i) for i in list(board[:, COLUMN_COUNT//2])]
	center_count = center_array.count(piece)
	score += center_count * 3

	# Score horizontal
	for r in range(ROW_COUNT):
		row_array = [int(i) for i in list(board[r,:])]
		for c in range(COLUMN_COUNT-3):
			window = row_array[c:c+WINDOW_LENGTH] # Our window length is 4
			score += evaluate_window(window, piece)

	# Score vertical
	for c in range(COLUMN_COUNT):
		col_array = [int(i) for i in list(board[:,c])]
		for r in range(ROW_COUNT-3):
			window = col_array[r:r+WINDOW_LENGTH]
			score += evaluate_window(window, piece)

	# Score positively sloped diagonal
	for r in range(ROW_COUNT-3):
		for c in range(COLUMN_COUNT-3):
			window = [board[r+i][c+i] for i in range(WINDOW_LENGTH)]
			score += evaluate_window(window, piece)
	
	# Score negatively sloped diagonal
	for r in range(ROW_COUNT-3):
		for c in range(COLUMN_COUNT-3):
			window = [board[r+3-i][c+i] for i in range(WINDOW_LENGTH)]
			score += evaluate_window(window, piece)

	return score

def is_terminal_node(board):
    # Terminal nodes describe where branching stops,
	# such as a win or no more possible locations

	return winning_move(board, PLAYER_PIECE) \
		or winning_move(board, AI_PIECE) \
		or len(get_valid_locations(board)) == 0
  
def get_valid_locations(board):
    # Extension of is_valid_location returning an iterable
	valid_locations = []
	for col in range(COLUMN_COUNT):
		if is_valid_location(board, col):
			valid_locations.append(col)
	return valid_locations

def pick_best_move(board, piece):

	valid_locations = get_valid_locations(board)
	best_score = -10000
	best_col = random.choice(valid_locations)
	for col in valid_locations:
		row = get_next_open_row(board, col)
		temp_board = board.copy()
		drop_piece(temp_board, row, col, piece)
		score = score_position(temp_board, piece)
		if score > best_score:
			best_score = score
			best_col = col

	return best_col

Search can be stopped below any MIN node having a Beta value less than or equal to the Alpha value of any of its MAX node ancestors. 

In [9]:
def minimax(board, depth, alpha, beta, maximizingPlayer, prune=True):
	valid_locations = get_valid_locations(board) # Determine all possible movements that can be made this turn
	is_terminal = is_terminal_node(board) # Determine whether current state is terminal

	# Upon reaching end of search depth or winning board
	if depth == 0 or is_terminal:
		if is_terminal:
			if winning_move(board, AI_PIECE):
				return (None, 1e10)
			elif winning_move(board, PLAYER_PIECE):
				return (None, -1e10)
			else: # Game is over, no more valid moves
				return (None, 0)
		else: # Depth is zero
			return (None, score_position(board, AI_PIECE))

	# Performing max step
	if maximizingPlayer:
		value = -math.inf
		column = random.choice(valid_locations) # Choose this column when new_score <= value

		# Branch on each possible move
		for col in valid_locations:
			row = get_next_open_row(board, col)
			# Copy board here to avoid modifying actual board
			b_copy = board.copy()
			drop_piece(b_copy, row, col, AI_PIECE)
			
			# Extend search to our new board with piece dropped
			new_score = minimax(b_copy, depth-1, alpha, beta, False)[1]
			
			# Choose move with highest score
			if new_score > value:
				value = new_score
				column = col
			
			# Perform alpha-beta pruning
			if prune:
				alpha = max(alpha, value)
				if alpha >= beta:
					break
		return column, value

	# Performing min step
	else:
		value = math.inf
		column = random.choice(valid_locations)
		for col in valid_locations:
			row = get_next_open_row(board, col)
			b_copy = board.copy()
			drop_piece(b_copy, row, col, PLAYER_PIECE)
			new_score = minimax(b_copy, depth-1, alpha, beta, True)[1]
			if new_score < value:
				value = new_score
				column = col

   			# Perform alpha-beta pruning
			if prune:
				beta = min(beta, value)
				if alpha >= beta:
					break
		return column, value

Game loop:

In [ ]:
board = create_board()
game_over = False

pygame.init()

SCREEN = pygame.display.set_mode((S_WIDTH, S_HEIGHT))
draw_board(board)
pygame.display.update()

myfont = pygame.font.SysFont("monospace", 75)

turn = random.randint(PLAYER, AI)

#PRUNE = True

while not game_over:
	for event in pygame.event.get():
		if event.type == pygame.QUIT:
			pygame.quit()
			sys.exit()

		if event.type == pygame.MOUSEMOTION:
			pygame.draw.rect(SCREEN, BLACK, (0,0, S_WIDTH, SQUARESIZE))
			posx = event.pos[0]
			if turn == PLAYER:
				pygame.draw.circle(SCREEN, RED, (posx, int(SQUARESIZE/2)), RADIUS)

		pygame.display.update()

		if event.type == pygame.MOUSEBUTTONDOWN:
			pygame.draw.rect(SCREEN, BLACK, (0,0, S_WIDTH, SQUARESIZE))
			#print(event.pos)
			# Ask for Player 1 Input
			if turn == PLAYER:
				posx = event.pos[0]
				col = int(math.floor(posx/SQUARESIZE))

				if is_valid_location(board, col):
					row = get_next_open_row(board, col)
					drop_piece(board, row, col, PLAYER_PIECE)

					if winning_move(board, PLAYER_PIECE):
						label = myfont.render("Player 1 wins!!", 1, RED)
						SCREEN.blit(label, (40,10))
						game_over = True

					turn += 1
					turn = turn % 2
					draw_board(board)

	# Ask for Player 2 Input
	if turn == AI and not game_over:				
		start = time.time()
		# Alpha and beta here have infinite scores because they haven't been established yet
		col, minimax_score = minimax(board, depth=5, alpha=-math.inf, beta=math.inf, maximizingPlayer=True, prune=False)

		finish = time.time() - start
		print("Time taken: %.2f seconds" % finish)
		if is_valid_location(board, col):
    		
			row = get_next_open_row(board, col)
			drop_piece(board, row, col, AI_PIECE)

			if winning_move(board, AI_PIECE):
				label = myfont.render("Player 2 wins!!", 1, YELLOW)
				SCREEN.blit(label, label.get_rect(center=(S_WIDTH/2, 50)))
				game_over = True

			turn += 1
			turn = turn % 2
			draw_board(board)

	if game_over:
		pygame.time.wait(3000)
		pygame.quit()